![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación de librerías
%matplotlib inline
import pandas as pd

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

data.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [3]:
# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data = data.drop(['Model'], axis=1)

# Visualización dataset
data.head()

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
7,21995,2014,6480,0,0,0,1,0,0,0
11,13995,2014,39972,0,0,0,0,1,0,0
167,17941,2016,18989,0,0,0,0,0,1,0
225,12493,2014,51330,0,0,0,1,0,0,0
270,7994,2007,116065,0,1,0,0,0,0,0


In [4]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['Price']
X = data.drop(['Price'], axis=1)

In [5]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el RMSE y MAE del modelo en el set de test.

In [ ]:
# Celda 1
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
def build_tree(X, y, max_depth, min_samples_split, depth=0):
    n_samples, n_features = X.shape
    
    # Comprobar si hemos alcanzado la profundidad máxima o si tenemos menos muestras que las muestras mínimas para dividir
    if depth >= max_depth or n_samples < min_samples_split:
        return np.mean(y)

    # Encontrar la característica y el umbral que minimizan el error
    best_loss = np.inf
    best_feature, best_threshold = None, None
    for feature in range(n_features):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            y_left = y[X[:, feature] < threshold]
            y_right = y[X[:, feature] >= threshold]
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            loss = np.sum((y_left - np.mean(y_left))**2) + np.sum((y_right - np.mean(y_right))**2)
            if loss < best_loss:
                best_loss = loss
                best_feature = feature
                best_threshold = threshold

    # Verificar si no pudimos encontrar una división que reduzca el error
    if best_feature is None:
        return np.mean(y)

    # Dividir los datos en nodos secundarios izquierdo y derecho
    left_indices = X[:, best_feature] < best_threshold
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[~left_indices], y[~left_indices]

    # Crear los nodos secundarios izquierdo y derecho
    left = build_tree(X_left, y_left, max_depth, min_samples_split, depth+1)
    right = build_tree(X_right, y_right, max_depth, min_samples_split, depth+1)

    # Crear el nodo actual
    return {'feature': best_feature, 'threshold': best_threshold, 'left': left, 'right': right}

In [ ]:
# Construcción del árbol
max_depth = 5
min_samples_split = 10
tree = build_tree(X.values, y.values, max_depth, min_samples_split)

print(tree)

In [ ]:
def predict_sample(x, tree):
    # Función auxiliar para predecir una muestra utilizando el árbol de decisión
    if x[tree['feature']] < tree['threshold']:
        if isinstance(tree['left'], dict):
            return predict_sample(x, tree['left'])
        else:
            return tree['left']
    else:
        if isinstance(tree['right'], dict):
            return predict_sample(x, tree['right'])
        else:
            return tree['right']

def predict(X, tree):
    # Función para predecir un conjunto de datos utilizando el árbol de decisión
    y_pred = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        y_pred[i] = predict_sample(X[i], tree)
    return y_pred

# Predecir los valores de y utilizando el conjunto de prueba y el árbol de decisión
y_pred = predict(X_test, tree)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calcular RMSE y MAE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
reg_tree = DecisionTreeRegressor(max_depth=3, random_state=42)
reg_tree.fit(X_train, y_train)

In [ ]:
y_pred = reg_tree.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
# Calcular el RMSE y MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print('RMSE:', rmse)
print('MAE:', mae)

In [ ]:
import matplotlib.pyplot as plt
# Ajustar una línea de tendencia
z = np.polyfit(y_test, y_pred, 1)
p = np.poly1d(z)

# Graficar la dispersión y la línea de tendencia
plt.scatter(y_test, y_pred)
plt.plot(y_test, p(y_test), "r--")
plt.xlabel('Precios reales')
plt.ylabel('Precios predichos')
plt.title('RMSE')
plt.show()

### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de clasificación y comenten sobre el desempeño del modelo.

In [7]:
# Celda 2

# Se crea un arreglo de 1 a 20
np.random.seed(1)

# Impresión de arreglo y muestreo aleatorio
nums = np.arange(1, 21)
print('Arreglo:', nums)
print('Muestreo aleatorio: ', np.random.choice(a=nums, size=20, replace=True))


In [ ]:
# Creación de 10 muestras de bootstrap 
np.random.seed(123)

n_samples = X_train.shape[0]
n_B = 10

samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
samples

In [ ]:
# Construcción un árbol de decisión para cada muestra boostrap

from sklearn.tree import DecisionTreeRegressor

# Definición del modelo usando DecisionTreeRegressor de sklearn
treereg = DecisionTreeRegressor(max_depth=None, random_state=123)

In [ ]:
# DataFrame para guardar las predicciones de cada árbol
y_pred = pd.DataFrame(columns=['Price'])

# Entrenamiento de un árbol sobre cada muestra boostrap y predicción sobre los datos de test
for i, sample in enumerate(samples):
    X_train = X_train.iloc[sample, 0:]
    y_train = y_train.iloc[sample]
    treereg.fit(X_train, y_train)
    y_pred_i = treereg.predict(X_test)
    y_pred['Price' + str(i+1)] = y_pred_i

In [ ]:
y_pred = y_pred.drop(['Price'], axis=1)

In [ ]:
# Desempeño de cada árbol
for i in range(n_B):
    print('Árbol ', i, 'tiene un error: ', np.sqrt(mean_squared_error(y_pred.iloc[:,i], y_test)))

In [ ]:
# Predicciones promedio para cada obserbación del set de test
y_pred.mean(axis=1)

In [ ]:
# Error al promediar las predicciones de todos los árboles
np.sqrt(mean_squared_error(y_test, y_pred.mean(axis=1)))

### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de clasificación y el parámetro `max_features` igual a `log(n_features)` y comenten sobre el desempeño del modelo.

In [27]:
from sklearn.ensemble import BaggingRegressor
from math import log
import numpy as np

features = X.columns
n_features=len(features)

clfBag = BaggingRegressor(n_estimators=10, max_features=max(1, int(log(n_features))))
clfBag.fit(X_train, y_train)


BaggingRegressor(max_features=2)

In [28]:
prediccionclfBag = clfBag.predict(X_test)
MSE_clfBag = np.average(np.square(prediccionclfBag-y_test))
print('\nMSE Bagging:')
print(MSE_clfBag)


MSE Bagging:
5535694.2886524685


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para clasificación  y comenten sobre el desempeño del modelo.

In [31]:
from sklearn.ensemble import RandomForestRegressor

clfRF = RandomForestRegressor(n_estimators=100, max_features=n_features, random_state=1, n_jobs=-1)
clfRF.fit(X_train, y_train)

RandomForestRegressor(max_features=9, n_jobs=-1, random_state=1)

In [32]:
prediccionclfRF = clfRF.predict(X_test)
MSE_clfRF = np.average(np.square(prediccionclfRF-y_test))
print('\nMSE Bagging:')
print(MSE_clfRF)


MSE Bagging:
3083089.1739264703


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para clasificación, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [10]:
# Celda 5


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de clasificación con la librería sklearn y comenten sobre el desempeño del modelo.

In [11]:
# Celda 6


### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para clasificación, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [12]:
# Celda 7


### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [13]:
# Celda 8
